# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd
import numpy as np

#System Libraries
import os, glob
import shutil
from datetime import datetime, timedelta

#Investing library
import investpy

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim

#Librarie to save the plots to html object
import panel as pn

hv.extension('bokeh')

# Initial Setup

In [ ]:
data = pd.read_excel('Countries.xlsx')
data = data.set_index('Maturities/Countries')

countries = data.index
maturities = data.columns

In [ ]:
#Create a Data Folder
Data_folder = os.path.abspath(os.getcwd() +'/Data/')
if not os.path.exists(Data_folder):
    os.makedirs(Data_folder)

#Clean older files and folders in the Data Folder
filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    shutil.rmtree(os.path.join(Data_folder, f), ignore_errors=True)

filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    os.remove(os.path.join(Data_folder, f))

#Create Folder to Store Full Curves for each country
full_curves_folder = os.path.join(Data_folder, 'Full Curves')
if not os.path.exists(full_curves_folder):
    os.makedirs(full_curves_folder)
    
#Create Folder to Store Data for each data point (maturity) for each country - Each Country will have a folder, see next
individual_points_folder = os.path.join(Data_folder, 'Individual Points')
if not os.path.exists(individual_points_folder):
    os.makedirs(individual_points_folder)
    
#Create New Folder for each Country
for country in countries:
    country_folder = os.path.join(individual_points_folder, country)
    if not os.path.exists(country_folder):
        os.makedirs(country_folder)

In [ ]:
#Setting the Initial and Last Date for Data Extract
today = datetime.now()
beginning_date = today - timedelta(days=365)
today = today.strftime('%d/%m/%Y')
beginning_date = beginning_date.strftime('%d/%m/%Y')

In [ ]:
#Loop to download historical data for each maturity for each country
##Loop through countries
for country in countries:
    try:
        #Defining the Country Folder (Special treatment for country names finishing with '.' due to folders names not finishing in '.')
        if country[-1] == '.':
            country_folder = os.path.join(individual_points_folder, country[:-1])
        else: country_folder = os.path.join(individual_points_folder, country)

        investing_symbols = data[data.index==country].iloc[0].tolist()
        
        #Loop through maturities
        for symbol in investing_symbols:
            try:       
                table = investpy.get_bond_historical_data(bond=symbol, from_date=beginning_date, to_date=today)
                table = table.rename(columns={"Close": symbol})
                table = table[symbol]
                table.to_csv(os.path.join(country_folder,symbol)+'_Table.csv', header = True)
            except:
                continue
    except:
        continue

In [ ]:
#Joining all files from different maturities in one file
for country in countries:
    try:
        if country[-1] == '.':
            country_folder = os.path.join(individual_points_folder, country[:-1])
        else: country_folder = os.path.join(individual_points_folder, country)

        all_files = glob.glob(country_folder + "/*.csv")

        full_df = pd.read_csv(all_files[0])
        for file in all_files[1:]:
            df = pd.read_csv(file, index_col=None)
            full_df = full_df.merge(df, on='Date', how='left')

        #Creating a Final DataFrame with All original Maturities - to mantain consistency along different countries
        #Note that different Governments issue bonds with different Maturities
        final_df = pd.DataFrame(columns=maturities)
        for maturity in maturities:
            try:
                final_df[maturity]=full_df[country+" "+maturity]
            except:
                final_df[maturity] = np.NaN

        final_df = final_df.fillna(method='bfill')

        #Limiting DataFrame to Include just the 4 Periods: 'Today','1Month Ago','3Months Ago','1Year Ago'
        final_df = final_df.iloc[[-1, -30,-90,0],:]
        final_df['Period'] = ['Today','1Month Ago','3Months Ago','1Year Ago']
        final_df = final_df.set_index('Period')

        final_df.to_csv(os.path.join(full_curves_folder,country)+'_Table.csv', header = True)
    except:
        continue

# Creating the Graphs

## US Yield Curve Graphs

In [ ]:
#Get the Data for current U.S. Yield Curve
country = 'U.S.'
US_table = pd.read_csv(os.path.join(full_curves_folder,country)+'_Table.csv')
US_YC_current = US_table[US_table['Period']=='Today'].set_index('Period')
US_YC_current = US_YC_current.T.interpolate('linear')

In [ ]:
#Generate Plot for current U.S. Yield Curve
curve_US_Current = hv.Curve(US_YC_current)
curve_US_Current = curve_US_Current.opts(opts.Curve(tools=['hover'], show_grid=True,
                              width=700, xlabel= 'Maturity', ylabel = 'Yield %', title = 'U.S. Yield Curve'))

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(curve_US_Current)
p.save('US_Yield_Curve_current.html', embed = True)

### US Yield Curve at Different Dates Graph

In [ ]:
#Get the Data for U.S. Yield Curve at different dates
country = 'U.S.'
US_table = pd.read_csv(os.path.join(full_curves_folder,country)+'_Table.csv')
US_YC = US_table.set_index('Period')
US_YC = US_YC.T.interpolate('linear')

In [ ]:
def getCurves(n):
    for column in US_YC.columns:
        curve = hv.Curve(US_YC[column], label = column)
        yield curve

target_curves  = []        
for curve in getCurves(10):
    # Without relabel, the curve somehow shares the ranging properties. opts with clone=True doesn't help either.
    tgt = curve.opts(opts.Curve(tools=['hover'], width=700))
    target_curves.append(tgt)     

#Overlay the source and target curves 
curves_US = hv.Overlay(target_curves).relabel('').opts(ylabel='Yield %', legend_position='top', height=400,
                                                              title = 'U.S. Yield Curves at Different Times')

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(curves_US)
p.save('US_Yield_Curves_Historical.html', embed = True)

## Multiple Yield Curves

In [ ]:
#For multiple Countries we need to create a DataFrame with 3-dimensions (country, maturity, date)
    #-> to do this we create a dictionary and include each DataFrame with the assigned dictionary key being the Index
    
multi_df = dict()
for country in countries:
    try:
        #For each of the Country will do the following procedure

        country_table = pd.read_csv(os.path.join(full_curves_folder,country)+'_Table.csv')
        country_table = country_table.set_index('Period')
        country_table_T = country_table.T
        country_table_T = country_table_T.interpolate(method='linear', limit_direction = 'both', limit_area = 'inside')
        multi_df.update({country: country_table_T})
    except:
        continue

In [ ]:
#Include Dropdown List
def load_countries(Country): 
    def getCurves(n):
        for column in multi_df[Country].columns:
            curve = hv.Curve(multi_df[Country][column], label = column)
            yield curve

    source_curves, target_curves  = [], []
    for curve in getCurves(10):
        # Without relabel, the curve somehow shares the ranging properties. opts with clone=True doesn't help either.
        tgt = curve.opts(opts.Curve(tools=['hover'], width=700))
        target_curves.append(tgt)     

    #Overlay the source and target curves 
    overlaid_plot_tgt = hv.Overlay(target_curves).relabel('').opts(ylabel='Yield %', legend_position='top_left', 
                                                                   background_fill_alpha  = 0, height=400)

    return overlaid_plot_tgt
        
countries_name = multi_df.keys()

dmap = hv.DynamicMap(load_countries, kdims='Country').redim.values(Country=countries_name)

dmap = dmap.opts(framewise=True)

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(dmap)
p.save('Multiple_Yield_Curves.html', embed = True) 

## Save Table to html format

In [ ]:
#For multiple Countries we need to create a DataFrame with 3-dimensions (country, maturity, date)
    #-> to do this we create a dictionary and include each DataFrame with the assigned dictionary key being the Index

#*** Similar to the one above but without Interpolation and Transpose ***
multi_df = dict()
for country in countries:
    try:
        #For each of the Country will do the following procedure

        country_table = pd.read_csv(os.path.join(full_curves_folder,country)+'_Table.csv')
        multi_df.update({country: country_table})
    except:
        continue

In [ ]:
#Instantiate a Table and define the Table Generation dynamics
##Note that the "if's" were introduced to account for the "All" values in Sectors and Industries as no stock has this value 
def load_countries(Country):
    table = hv.Table(multi_df[Country])  
    table = table.opts(opts.Table(width=950, height=150, selectable = True, index_position = None))
    return table

countries_name = multi_df.keys()

#Instantiate the DynamicMap function, so to generate the Table defined with the Widgets for Sectors and Industries
dmap = hv.DynamicMap(load_countries, kdims=['Country']).redim.values(Country=countries_name)
dmap = dmap.opts(framewise=True)

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(dmap,  widget_location='top_left')
p.save('Yield_Curves_table.html', embed = True)

# Extra: Examples for Types of Yield Curves 

In [ ]:
YC_examples = pd.DataFrame(data={'Maturity': ['3M','6M','1Y', '2Y','5Y','7Y','10Y','15Y','20Y','30Y'], 
                                 'Normal': [0.2, 0.6, 1, 1.3, 1.5, 1.7, 1.8, 1.9, 2, 2.05],
                                'Flat': [1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5],
                                'Inverted':[2.05, 2, 1.9, 1.8, 1.7, 1.5, 1.3, 1, 0.6, 0.2],
                                'Humped':[0.8, 0.9, 0.7, 0.6, 0.7, 0.8, 0.9, 1, 1.2, 1.3]}).set_index('Maturity')

In [ ]:
Normal_YC = hv.Curve(YC_examples['Normal'], label='Normal Yield Curve').opts(ylim=(0, 3), ylabel='Yield %')
Inverted_YC = hv.Curve(YC_examples['Inverted'], label='Inverted Yield Curve').opts(ylim=(0, 3), ylabel='Yield %')
Flat_YC = hv.Curve(YC_examples['Flat'], label='Flat Yield Curve').opts(ylim=(0, 3), ylabel='Yield %')
Humped_YC = hv.Curve(YC_examples['Humped'], label='Humped Yield Curve').opts(ylim=(0, 3), ylabel='Yield %')

In [ ]:
YC_types = hv.Layout(Normal_YC + Inverted_YC + Flat_YC + Humped_YC).cols(2)

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(YC_types)
p.save('Yield_Curves_Types.html', embed = True)